In [2]:
!pip install pandas

You should consider upgrading via the '/Users/fabioteichmann/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv('data/train.csv')

In [5]:
# Count occurrences of `ImageId` in df
df['count'] = df.ImageId.apply(lambda x: df['ImageId'].value_counts()[x])

In [6]:
df.head(10)

,ImageId,ClassId,EncodedPixels,count
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,1
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...,1
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...,1
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...,1
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...,1
5,0025bde0c.jpg,3,8458 14 8707 35 8963 48 9219 71 9475 88 9731 8...,2
6,0025bde0c.jpg,4,315139 8 315395 15 315651 16 315906 17 316162 ...,2
7,002af848d.jpg,4,290800 6 291055 13 291311 15 291566 18 291822 ...,1
8,002fc4e19.jpg,1,146021 3 146275 10 146529 40 146783 46 147038 ...,2
9,002fc4e19.jpg,2,145658 7 145901 20 146144 33 146386 47 146629 ...,2


In [21]:
def count_defect_pixels(EncodedPixels):
    sum = 0
    for idx, pixel in enumerate(EncodedPixels.split()):
        if idx%2 != 0:
            sum += int(pixel)

    return sum

In [20]:
df['DefectSizePixel'] = df.EncodedPixels.apply(lambda x: count_defect_pixels(x))
df.head()

,ImageId,ClassId,EncodedPixels,count,DefectSizePixel
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,1,4396
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...,1,6897
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...,1,8319
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...,1,69357
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...,1,4851


In [23]:
df.groupby('ClassId')['DefectSizePixel'].mean().reset_index()

,ClassId,DefectSizePixel
0,1,4361.347826
1,2,3378.425101
2,3,25496.485243
3,4,34373.997503


It becomes clear that, on average, defects of `ClassId` 3 and 4 are much bigger in size.

In [28]:
!pip3 install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 4.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [43]:
# Calculate fraction of defect relative to the picture size
import cv2

def get_image_height(image):
    image = cv2.imread('data/train_images/' + image) # read an image by its path
    return image.shape[0] # image.shape returns a tupel (height, width, depth)

def get_image_width(image):
    image = cv2.imread('data/train_images/' + image)
    return image.shape[1]

def get_image_depth(image):
    image = cv2.imread('data/train_images/' + image)
    return image.shape[2]

In [ ]:
# Create columns for `Height`, `Width`, and `Depth`
df['Height'] = df.ImageId.apply(lambda x: get_image_height(x))
df['Width'] = df.ImageId.apply(lambda x: get_image_width(x))
df['Depth'] = df.ImageId.apply(lambda x: get_image_depth(x))

df['DefectSizePixelRelative'] = df.DefectSizePixel / (df.Height * df.Width)

In [37]:
df.head()

,ImageId,ClassId,EncodedPixels,count,DefectSizePixel,Height,Width,Depth,DefectPixelSizeRelative
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,1,4396,256,1600,3,0.010732
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...,1,6897,256,1600,3,0.016838
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...,1,8319,256,1600,3,0.020310
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...,1,69357,256,1600,3,0.169329
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...,1,4851,256,1600,3,0.011843


In [38]:
# Check whether we have different dimensions for pictures
df.describe()

,ClassId,count,DefectSizePixel,Height,Width,Depth,DefectPixelSizeRelative
count,7095.000000,7095.000000,7095.000000,7095.0,7095.0,7095.0,7095.000000
mean,2.825229,1.121494,23056.669626,256.0,1600.0,3.0,0.056291
std,0.789279,0.329302,34957.878108,0.0,0.0,0.0,0.085346
min,1.000000,1.000000,115.000000,256.0,1600.0,3.0,0.000281
25%,3.000000,1.000000,4251.000000,256.0,1600.0,3.0,0.010378
50%,3.000000,1.000000,10341.000000,256.0,1600.0,3.0,0.025247
75%,3.000000,1.000000,26791.000000,256.0,1600.0,3.0,0.065408
max,4.000000,3.000000,368240.000000,256.0,1600.0,3.0,0.899023


In [42]:
df.groupby('ClassId')['DefectSizePixelRelative'].mean().reset_index()

,ClassId,DefectSizePixelRelative
0,1,0.010648
1,2,0.008248
2,3,0.062247
3,4,0.083921


Relative defect size on average reaches up to 8.4% for defect class 4. As we can see above, however, the maximum coverage of a defect may reach **up to 89.9%** of the respective image.

In [7]:
df_duplicates = df.query('count > 1')
df_duplicates.info()

In [8]:
df.groupby('ClassId').count()

,ImageId,EncodedPixels,count
ClassId,,,
1,897,897,897
2,247,247,247
3,5150,5150,5150
4,801,801,801


--> there is a strong **imbalance** in the representation of different defect type:
- error 1 = 0.135 %
- error 2 = 0.037 %
- error 3 = 0.773 % of all images
- error 4 = 0.120 %

In [ ]:
# Dimension of our pictures
1600*256

In [ ]:
print(f"The training data has {df.ImageId.nunique()} unique image IDs")

### To-Dos Daniela

In [ ]:
df_classes=df_duplicates.groupby('ImageId')['ClassId'].sum().reset_index()
df_classes.rename(columns={'ClassId': 'SumClasses'}, inplace=True)
df_classes.head()

In [ ]:
sns.histplot(df_classes)

In [ ]:
pivot1 = pd.pivot_table(df_duplicates, index='ImageId', columns='ClassId', values='count', aggfunc = 'count', fill_value=0)
pivot1

In [ ]:
pivot2 = pd.pivot_table(df_duplicates, index='ClassId', columns='ClassId', values='ImageId', aggfunc = 'count', fill_value=0)
pivot2

In [ ]:
pivot1[1]=pivot1[1].astype(int)

In [ ]:

ax = df.T.plot(kind='hist', ylabel='ClassId')

In [ ]:
result = pd.merge(df_classes, pivot1, on="ImageId")
result.head()

In [ ]:
result.info

In [ ]:
#df.astype(int).T.dot(df)
result.rename(columns={1: '1',2: '2',3: '3',4: '4'}, inplace=True)
#temp = result.columns.values.tolist()
temp = result[['1','2','3','4']]
a= temp.T.dot(temp)
a
#temp